In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.impute import KNNImputer

# Data Cleaning

In [104]:
train = pd.read_csv('train_subset.csv')

In [105]:
test = pd.read_csv('test_subset.csv')

In [110]:
test

,Unnamed: 0,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961b9232e2e0f1fad84b8,2017-07-03,0.9105192015197214,0.0033230903641159,0.002007364114837,0.8130097415205554,0.0004830524470232578,Unnamed: 7,0.0085777884477019,...,Unnamed: 180,Unnamed: 181,Unnamed: 182,0.0050550568844862,0.0033865326346197,0.0063811991547401,Unnamed: 186,0.0090705763766117,0.0004515262697018397,0.009092919156372
0,0,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961...,2017-08-03,0.916596,0.002949,0.007152,0.810892,0.007676,NaN,0.004424,...,NaN,NaN,NaN,0.009045,0.005062,0.008398,NaN,0.009962,0.009318,0.003389
1,1,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961...,2017-09-02,0.913374,0.000478,0.002182,0.815958,0.000189,NaN,0.003844,...,NaN,NaN,NaN,0.002448,0.006637,0.002498,NaN,0.002704,0.009881,0.006629
2,2,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961...,2017-10-03,0.907133,0.009843,0.009453,0.819477,0.008656,NaN,0.001826,...,NaN,NaN,NaN,0.008992,0.002036,0.000650,NaN,0.007807,0.007089,0.009498
3,3,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961...,2017-11-03,0.871219,0.009839,0.000488,0.814127,0.003975,NaN,0.002952,...,NaN,NaN,NaN,0.000094,0.006466,0.006941,NaN,0.000021,0.009241,0.008565
4,4,00f1aea27e2b2e8504b3c0e3c691e1ed60b16a9d6b5961...,2017-12-02,0.944125,0.000807,0.009617,0.814131,0.001435,NaN,0.006016,...,NaN,NaN,NaN,0.001964,0.001842,0.006457,NaN,0.004139,0.000044,0.000650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,016661b2275fbd12b66a6a44d5274b3ce8a46b9bacdc01...,2017-05-16,0.968773,0.682007,0.033532,1.004179,0.005582,0.068700,0.001663,...,NaN,NaN,NaN,0.000122,0.002820,0.002444,NaN,0.006658,0.003938,0.005537
9996,9996,016661b2275fbd12b66a6a44d5274b3ce8a46b9bacdc01...,2017-06-01,0.994103,0.237832,0.028541,1.008582,0.002640,0.055308,0.004520,...,NaN,NaN,NaN,0.001989,0.006962,0.000827,NaN,0.003828,0.008829,0.000494
9997,9997,016661b2275fbd12b66a6a44d5274b3ce8a46b9bacdc01...,2017-07-20,0.971680,0.802516,0.037812,1.007390,0.006115,0.026462,0.007735,...,NaN,NaN,NaN,0.004796,0.005723,0.002804,NaN,0.004543,0.004120,0.000063
9998,9998,016661b2275fbd12b66a6a44d5274b3ce8a46b9bacdc01...,2017-08-24,0.990912,0.009236,0.019213,1.002660,0.005935,-0.035923,0.007416,...,NaN,NaN,NaN,0.003087,0.009291,0.008839,NaN,0.000551,0.006786,0.007417


In [106]:
y = pd.read_csv('train_labels.csv')

In [107]:
# merge response + predictors in train data
train = pd.merge(train, y)

In [108]:
train_filtered = train.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

In [109]:
test_filtered = test.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

KeyError: "['customer_ID', 'S_2', 'D_63', 'D_64'] not found in axis"

In [88]:
# storing missing values for each column in train
val_train = train_filtered.isna().sum()

# storing missing values for each column in test
val_test = test_filtered.isna().sum()

In [89]:
# creating dataframe to store how many values are missing in each column for both train and test
na_df = pd.DataFrame(columns = train_filtered.columns)
na_df.loc['Train'] = val_train
na_df.loc['Test'] = val_test

In [90]:
# identify predictors with large number of missing values 
col_missing = []
for col in na_df.columns:
    if na_df.loc['Train',col]/10000 > .8:
        col_missing.append(col)

In [91]:
# drop predictors with more than 80% missing values
train_filtered = train_filtered.drop(col_missing,axis=1)

In [92]:
test_filtered = test_filtered.drop(col_missing,axis=1)

In [93]:
imputer = KNNImputer(n_neighbors=1)

In [94]:
train_filtered = pd.DataFrame(imputer.fit_transform(train_filtered),columns = train_filtered.columns)

In [95]:
test_filtered = pd.DataFrame(imputer.fit_transform(test_filtered),columns = test_filtered.columns)

In [96]:
train_filtered.to_csv('train_clean.csv')

In [97]:
test_filtered.to_csv('test_clean.csv')

# Bagging

In [ ]:
# Separating the predictors and response
y = train_filtered.target
X = 